# <strong><font color=#A52A2A></span> NLP_HW4 </strong>
Dr Asgari - Spring 2023
<br>
98101566 -> Mohammadreza Daviran
<br>
98170668 -> Sara Azarnoush
<br>
98171007 -> Nona Ghazizadeh
<br>
<strong><font color=#A52A2A> Named Entity Recognition (NER)

# <font color='red'> Table of content
-------

 - Intoduction
 - Requirements
    - Download
    - Import
 - Prepare
   - Model
   - Data
 - Labels
    - Check
    - Translate
 - Predict
 - Calculate
 - Evaluation


# <font color='red'> Introduction </font>
-------

In this project, we extract information to locate and classify named entities mentioned in unstructured text into pre-defined categories including \\
Person (PER), Location (LOC), Main location (mainLoc), Event (EVE), Date (DAT), Organization (ORG), Time (TIM), Facility (FAC), Money (MON), Percent (PCT), Product (PRO). \\
First, we import our requirements and download our dataset then preprocess data and translate our dataset labels to model labels and train our model. In the end, we check our true and predicted outputs and evaluate them. \\
More information is available in each section.

# <font color='red'> Requirements </font>
-------

Download and import requirements for this project

## <font color='green'> Download </font>

Download project requirements

*note: If you get an error uninstall and install numpy then restart the execution and run again.

In [ ]:
# !pip uninstall numpy

In [ ]:
!pip install numpy

In [ ]:
!pip install transformers
!pip install hazm
!pip install seqeval
!pip install keras-preprocessing

  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached huggingface_hub-0.16.3-py3-none-any.whl (268 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached safetensors-0.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.7/349.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=fd4d698739d9f6fd21531eff7c4e1707a314c71dd5e7aff1515ae79ed61e578b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


## <font color='green'> Import </font>

Import project requirements

In [ ]:
import os
import gc
import ast
import time
import hazm
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import transformers
from transformers import AutoTokenizer, AutoConfig
from transformers import AutoModelForTokenClassification
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# <font color='red'> Prepare </font>
-------

Download dataset and preprocess data and then use model.

## <font color='green'> Model </font>

Preprocess data and use bert-base-parsbert-ner-uncased model for classification.

In [ ]:
class NER:
    def __init__(self, model_name):
        self.normalizer = hazm.Normalizer()
        self.model_name = model_name
        self.config = AutoConfig.from_pretrained(self.model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForTokenClassification.from_pretrained(self.model_name)
        self.id2label = self.config.id2label

In [ ]:
model_name='HooshvareLab/bert-base-parsbert-ner-uncased'
ner_model = NER(model_name)

## <font color='green'> Data </font>

We create our dataset with arman+peyma+wikiann datasets and add some changes to it. It has 11 tags include: Person (PER), Location (LOC), Main location (mainLoc), Event (EVE), Date (DAT), Organization (ORG), Time (TIM),  Facility (FAC), Money (MON), Percent (PCT), Product (PRO).

Dataset available in:

https://drive.google.com/file/d/1w9ubKB-QOz1P99-HmusuWLM2akJBIjNs/view?usp=sharing



Download dataset from googledrive and unzip

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id': '1w9ubKB-QOz1P99-HmusuWLM2akJBIjNs'})
download.GetContentFile('ner-v2edited.zip')
!unzip ner-v2edited.zip

Archive:  ner-v2edited.zip
   creating: ner-edited/
   creating: ner-edited/ner/
  inflating: ner-edited/ner/ner.csv  
  inflating: ner-edited/ner/test.csv  
  inflating: ner-edited/ner/train.csv  
  inflating: ner-edited/ner/valid.csv  


Check file path and read dataset then get words and their lables

In [ ]:
def load_datasets(self, dataset_dir, **kwargs):
            ner_file_path = dataset_dir + 'test.csv'
            if not os.path.exists(ner_file_path):
                print(ner_file_path)
                exit(1)
            data = pd.read_csv(ner_file_path, delimiter="\t")
            sentences, sentences_tags = data['tokens'].values.tolist(), data['ner_tags'].values.tolist()
            sentences = [ast.literal_eval(ss) for ss in sentences]
            sentences_tags = [ast.literal_eval(ss) for ss in sentences_tags]
            print(f'test part:\n #sentences: {len(sentences)}, #sentences_tags: {len(sentences_tags)}')
            return sentences, sentences_tags

Print sentences and labels

In [ ]:
sentences_paw, labels_paw = load_datasets(ner_model, dataset_dir="./ner-edited/ner/")
print(len(sentences_paw), len(labels_paw))
print(sentences_paw[3])
print(labels_paw[3])

test part:
 #sentences: 6049, #sentences_tags: 6049
6049 6049
['تهران', 'به', '0', 'کیلومتر', 'بزرگراه', 'نیاز', 'دارد', '.']
['LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


# <font color='red'>  Labels </font>
-------

Get dataset and model labels compare them and translate dataset labels to model labels


In [ ]:
def check_label(self, labels):
        model_labels = self.config.label2id.keys()
        dataset_labels = set()
        for l in labels:
            dataset_labels.update(set(l))
        print("model labels:", model_labels)
        print("dataset labels:", dataset_labels)
        if list(set(dataset_labels) - set(model_labels)):
            return False
        return True

def resolve_label(labels, label_translation_map):
        for i, sentence_labels in enumerate(labels):
            for j, label in enumerate(sentence_labels):
                labels[i][j] = label_translation_map.get(label)
        return labels

In [ ]:
is_consistent = check_label(ner_model, labels_paw)
print(is_consistent)

model labels: dict_keys(['B-date', 'B-event', 'B-facility', 'B-location', 'B-money', 'B-organization', 'B-percent', 'B-person', 'B-product', 'B-time', 'I-date', 'I-event', 'I-facility', 'I-location', 'I-money', 'I-organization', 'I-percent', 'I-person', 'I-product', 'I-time', 'O'])
dataset labels: {'I-MON', 'I-FAC', 'B-DAT', 'B-EVE', 'I-PER', 'B-PCT', 'I-ORG', 'B-FAC', 'I-EVE', 'I-PRO', 'B-PRO', 'O', 'I-TIM', 'I-PCT', 'LOC', 'B-PER', 'I-DAT', 'ⅿainⅬoⅽ', 'B-ORG', 'B-MON', 'B-TIM'}
False


Translate datadet labels to model labels

In [ ]:
label_translate = {
    'B-MON': 'B-money',
    'I-MON': 'I-money',
    'B-ORG': 'B-organization',
    'I-ORG': 'I-organization',
    'LOC': 'B-location',
    'ⅿainⅬoⅽ': 'I-location',
    'B-PER': 'B-person',
    'I-PER': 'I-person',
    'B-DAT': 'B-date',
    'I-DAT': 'I-date',
    'B-TIM': 'B-time',
    'I-TIM': 'I-time',
    'B-FAC': 'B-facility',
    'I-FAC': 'I-facility',
    'B-PCT': 'B-percent',
    'I-PCT': 'I-percent',
    'B-EVE': 'B-event',
    'I-EVE': 'I-event',
    'B-PRO': 'B-product',
    'I-PRO': 'I-product',
    'O': 'O'
}

In [ ]:
labels_paw = resolve_label(labels_paw, label_translate)
is_consistent = check_label(ner_model, labels_paw)
print(is_consistent)

model labels: dict_keys(['B-date', 'B-event', 'B-facility', 'B-location', 'B-money', 'B-organization', 'B-percent', 'B-person', 'B-product', 'B-time', 'I-date', 'I-event', 'I-facility', 'I-location', 'I-money', 'I-organization', 'I-percent', 'I-person', 'I-product', 'I-time', 'O'])
dataset labels: {'B-percent', 'B-time', 'I-location', 'I-date', 'I-person', 'B-organization', 'I-percent', 'I-money', 'B-location', 'B-money', 'B-date', 'O', 'B-person', 'I-facility', 'B-facility', 'B-event', 'B-product', 'I-product', 'I-organization', 'I-event', 'I-time'}
True


# <font color='red'> Predict </font>
-------

Input a text and use model to locate and classify named entities mentioned in unstructured text into pre-defined categories   

In [ ]:
def predict(self, input_text, device, max_length):
        if not self.model or not self.tokenizer or not self.id2label:
            print('Something wrong has been happened!')
            return
        pt_batch = self.tokenizer(
            [self.normalizer.normalize(sequence) for sequence in input_text],
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        gc.collect()
        torch.cuda.empty_cache()
        if device.type != 'cpu':
            self.model.cuda()
        pt_batch = pt_batch.to(device)
        pt_outputs = self.model(**pt_batch)
        pt_predictions = torch.argmax(pt_outputs.logits, dim=-1)
        pt_predictions = pt_predictions.cpu().detach().numpy().tolist()
        output_predictions = []
        for i, sequence in enumerate(input_text):
            tokens = self.tokenizer.tokenize(self.tokenizer.decode(self.tokenizer.encode(sequence)))
            predictions = [(token, self.id2label[prediction]) for token, prediction in
                           zip(tokens, pt_predictions[i])]
            output_predictions.append(predictions)
        return output_predictions

Input a text to see result

In [ ]:
text = [
   "عبدالحسن برزیده دبیر هشتمین جشنواره فیلم دفاع مقدس و مدیرعامل انجمن سینمای انقلاب و دفاع  مقدس، روزیکشنبه با بیان این مطلب ، افزود: امسال انجمن سینمای انقلاب و دفاع مقدس در برگزاری جشنواره فیلم دفاع مقدس با بنیاد حفظ آثار و ارزشهای دفاع مقدس همکاری میکند و قرار است این روند در سالهای آینده نیز ادامه پیدا کند."
]

In [ ]:
output = predict(ner_model, text, device, ner_model.config.max_position_embeddings)
print(output)

[[('[CLS]', 'O'), ('عبدالحسن', 'B-person'), ('برزیده', 'I-person'), ('دبیر', 'O'), ('هشتمین', 'B-event'), ('جشنواره', 'I-event'), ('فیلم', 'I-event'), ('دفاع', 'I-event'), ('مقدس', 'I-event'), ('و', 'O'), ('مدیرعامل', 'O'), ('انجمن', 'B-organization'), ('سینمای', 'I-organization'), ('انقلاب', 'I-organization'), ('و', 'I-organization'), ('دفاع', 'I-organization'), ('مقدس', 'I-organization'), ('،', 'O'), ('روزیکشنبه', 'O'), ('با', 'O'), ('بیان', 'O'), ('این', 'O'), ('مطلب', 'O'), ('،', 'O'), ('افزود', 'O'), (':', 'O'), ('امسال', 'O'), ('انجمن', 'B-organization'), ('سینمای', 'I-organization'), ('انقلاب', 'I-organization'), ('و', 'I-organization'), ('دفاع', 'I-organization'), ('مقدس', 'I-organization'), ('در', 'O'), ('برگزاری', 'O'), ('جشنواره', 'B-event'), ('فیلم', 'I-event'), ('دفاع', 'I-event'), ('مقدس', 'I-event'), ('با', 'O'), ('بنیاد', 'B-organization'), ('حفظ', 'I-organization'), ('اثار', 'I-organization'), ('و', 'I-organization'), ('ارزشهای', 'I-organization'), ('دفاع', 'I-organiza

# <font color='red'> Calculate </font>
-------

Input text and labels and tokenized them then train our model and get predicted and true label.

In [ ]:
def calculate(self, input_text, input_labels, device, batch_size=4):
        if not self.model or not self.tokenizer or not self.id2label:
            print('Something wrong has been happened!')
            return
        print("len(input_text):", len(input_text))
        print("len(input_labels):", len(input_labels))
        c = 0
        max_len = 0
        tokenized_texts, new_labels = [], []
        for sentence, sentence_label in zip(input_text, input_labels):
            if type(sentence) == str:
                sentence = sentence.strip().split()
            if len(sentence) != len(sentence_label):
                print('Len of a sentence and label is not equal')
                return
            tokenized_words = self.tokenizer(sentence, padding=False, add_special_tokens=False).input_ids
            tokenized_sentence_ids, new_sentence_label = [], []
            for i, tokenized_word in enumerate(tokenized_words):
                tokenized_sentence_ids += tokenized_word
                new_sentence_label.extend([self.config.label2id.get(sentence_label[i])] * len(tokenized_word))
            max_len = max(max_len, len(tokenized_sentence_ids))
            tokenized_texts.append(tokenized_sentence_ids)
            new_labels.append(new_sentence_label)
            c += 1
            if c % 10000 == 0:
                print("c:", c)
        max_len = min(max_len, self.config.max_position_embeddings)
        print("max_len:", max_len)
        input_ids = pad_sequences(tokenized_texts, maxlen=max_len, dtype="long", value=self.config.pad_token_id,
                                  truncating="post", padding="post")
        del tokenized_texts
        input_labels = pad_sequences(new_labels, maxlen=max_len, value=self.config.label2id.get('O'), padding="post",
                                     dtype="long", truncating="post")
        del new_labels
        train_data = TensorDataset(torch.tensor(input_ids), torch.tensor(input_labels))
        data_loader = DataLoader(train_data, batch_size=batch_size)
        gc.collect()
        torch.cuda.empty_cache()
        if device.type != 'cpu':
            self.model.cuda()
        total_time = 0
        output_predictions = []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_input_ids, b_labels = batch
            b_input_ids = b_input_ids.to(device)
            b_labels = b_labels.to(device)
            with torch.no_grad():
                start = time.monotonic()
                outputs = self.model(b_input_ids, labels=b_labels)
                end = time.monotonic()
                total_time += end - start
                print(f'step {step}: {end - start}')
            b_predictions = torch.argmax(outputs.logits, dim=2)
            b_predictions = b_predictions.cpu().detach().numpy().tolist()
            b_labels = b_labels.cpu().detach().numpy().tolist()
            for i, sample in enumerate(b_input_ids):
                sample_input = list(sample)
                while sample_input[-1] == self.config.pad_token_id:
                    sample_input.pop()
                tokens = [self.tokenizer.decode([t]) for t in sample_input]
                sample_true_labels = [self.id2label[e] for e in b_labels[i][:len(sample_input)]]
                sample_predictions = [self.id2label[e] for e in b_predictions[i][:len(sample_input)]]
                output_predictions.append(
                    [(t, sample_true_labels[j], sample_predictions[j]) for j, t in enumerate(tokens)])
        print("total time:", total_time)
        return output_predictions


In [ ]:
inference_output = calculate(ner_model, sentences_paw, labels_paw, device, batch_size=256)

len(input_text): 6049
len(input_labels): 6049
max_len: 448
Start to evaluate test data ...
step 0: 0.028901369000095656
step 1: 0.013152546999890546
step 2: 0.019631026999832102
step 3: 0.013770911000165142
step 4: 0.014313252999954784
step 5: 0.01872096700003567
step 6: 0.013648431999854438
step 7: 0.02872460699995827
step 8: 0.014755558999922869
step 9: 0.01398725299986836
step 10: 0.01618858899996667
step 11: 0.018885328999886042
step 12: 0.013499891000037678
step 13: 0.013201222000134294
step 14: 0.02540642099984325
step 15: 0.013839246000088679
step 16: 0.0130064110001058
step 17: 0.013145461000021896
step 18: 0.022746528999959992
step 19: 0.013367232000064178
step 20: 0.012889507999943817
step 21: 0.013377641000033691
step 22: 0.018580743000029543
step 23: 0.014176819999875079
total time: 0.40191696799956844


Print each word, true catagory and our output

In [ ]:
for sample_output in inference_output[:5]:
  for token, true_label, predicted_label in sample_output:
    print('{}\t{}\t{}'.format(token, true_label, predicted_label))
  print()

همچنین	O	O
عملیات	O	O
لرزهنگاری	O	O
دوبعدی	O	O
نیز	O	O
با	O	O
فعالیت	O	O
مستمر	O	O
چهار	O	O
گروه	O	O
کاری	O	O
در	O	O
مناطقی	O	O
که	O	O
از	O	O
نظر	O	O
اکتشافی	O	O
مورد	O	O
نظر	O	O
بود	O	O
،	O	O
به	O	O
پایان	O	O
رسید	O	O
که	O	O
نتایج	O	O
ان	O	O
در	O	O
حال	O	O
بررسی	O	O
است	O	O
.	O	O

محدث	B-person	O
در	O	O
مورد	O	O
مشارکت	O	O
شرکتهای	O	O
خارجی	O	O
در	O	O
فعالیتهای	O	O
اکتشافی	O	O
کشور	O	O
گفت	O	O
:	O	O
تاکنون	O	O
چند	O	O
منطقه	O	O
اکتشافی	O	O
را	O	O
برای	O	O
مشارکت	O	O
و	O	O
سرمایهگذاری	O	O
شرکتهای	O	O
خارجی	O	O
اعلام	O	O
کردهایم	O	O
و	O	O
در	O	O
حال	O	O
مذاکره	O	O
با	O	O
طرفهای	O	O
خارجی	O	O
هستیم	O	O
و	O	O
انتظار	O	O
میرود	O	O
تا	O	O
اخر	O	O
امسال	O	O
بتوانیم	O	O
چند	O	O
قرارداد	O	O
را	O	O
نهایی	O	O
کنیم	O	O
.	O	O

مدیر	O	O
امور	B-organization	B-organization
اکتشاف	I-organization	I-organization
شرکت	I-organization	I-organization
ملی	I-organization	I-organization
نفت	I-organization	I-organization
فرو	O	O
##افتادگی	O	O
دزفول	B-location	B-location
و	O	O
منطقه	B-location	B-location
گسل	I-l

# <font color='red'> Evaluate </font>
-------

Compare true lables and predicted labels to find accuracy, prescision, recall and f1.

In [ ]:
def evaluate_results(labels, output_predictions):
        dataset_labels = set()
        for label in labels:
            dataset_labels.update(set(label))
        true_labels, predictions = [], []
        for sample_output in output_predictions:
            sample_true_labels = []
            sample_predicted_labels = []
            for token, true_label, predicted_label in sample_output:
                sample_true_labels.append(true_label)
                if predicted_label in dataset_labels:
                    sample_predicted_labels.append(predicted_label)
                else:
                    sample_predicted_labels.append('O')
            true_labels.append(sample_true_labels)
            predictions.append(sample_predicted_labels)

        print("Test Accuracy: {}".format(accuracy_score(true_labels, predictions)))
        print("Test Precision: {}".format(precision_score(true_labels, predictions)))
        print("Test Recall: {}".format(recall_score(true_labels, predictions)))
        print("Test F1-Score: {}".format(f1_score(true_labels, predictions)))


In [ ]:
evaluate_results(labels_paw, inference_output)

Test Accuracy: 0.9635493519441675
Test Precision: 0.7158046866935067
Test Recall: 0.6529190207156309
Test F1-Score: 0.6829172206628256
